In [ ]:
import os
import sys
import logging
import omegaconf
import glob
from pathlib import Path

sys.path.append('/explore/nobackup/people/jacaraba/development/tensorflow-caney')
sys.path.append('/explore/nobackup/people/jacaraba/development/vhr-cnn-chm')
sys.path.append('/home/pmontesa/code/icesat2')

import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
#from vhr_cnn_chm.model.vis import MAP_ATL08_FOLIUM
#from vhr_cnn_chm.model.utils import atl08_io
# from vhr_cnn_chm.model.cnn_regression_pipeline import CNNRegressionPipeline
from shapely.geometry import Point
from rasterio.plot import show

import os
import datetime
import pandas as pd
import geopandas as gpd
import branca.colormap as cm
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature

from shapely.geometry import shape
from geopandas import GeoDataFrame
from folium import (
    Map,
    TileLayer,
    GeoJson,
    LayerControl,
    features,
    Figure,
    CircleMarker,
    plugins
)
from rasterio.transform import from_origin
from matplotlib.colors import LinearSegmentedColormap

%matplotlib inline

In [ ]:
def atl08_io(
            atl08_csv_output_dir,
            year_search, do_pickle=True,
            filename_regex='ATL08*.csv'
        ):
    """
    Read all ATL08 from CSVs of a given year after extract_filter_atl08.py
    Write to a pickle file by year
    Return a geodataframe
    Modified from https://github.com/pahbs/geoscitools/blob/master/atl08lib.py
    """
    dir_pickle = atl08_csv_output_dir
    filename_regex = os.path.join(
        atl08_csv_output_dir, year_search, filename_regex)

    all_atl08_csvs = glob.glob(filename_regex, recursive=True)
    if len(all_atl08_csvs) < 1:
        logging.info(f"No ATL08 CSVs were found under {filename_regex}")
        return
    logging.info(f"Processing ATL08 CSV: {filename_regex}")

    atl08_gdf = pd.concat(
        (pd.read_csv(f) for f in all_atl08_csvs),
        sort=False, ignore_index=True)  # <--generator is (), list is []

    atl08_gdf = gpd.GeoDataFrame(
        atl08_gdf, geometry=gpd.points_from_xy(
            atl08_gdf.lon, atl08_gdf.lat), crs='epsg:4326')

    if do_pickle:
        # Pickle the file
        if year_search == "**":
            year_search = 'allyears'
        cur_time = time.strftime("%Y%m%d%H%M")
        out_pickle_fn = os.path.join(
            dir_pickle, f"atl08_{year_search}_filt_gdf_{cur_time}.pkl")
        atl08_gdf.to_pickle(out_pickle_fn)
    return atl08_gdf

In [ ]:
main_dir_serc = '/explore/nobackup/projects/ilab/projects/AIML_CHM/CNN_CHM/labels/atl08.005/serc'
main_dir_howland = '/explore/nobackup/projects/ilab/projects/AIML_CHM/CNN_CHM/labels/atl08.005/howland'
main_dir_tanana = '/explore/nobackup/projects/ilab/projects/AIML_CHM/CNN_CHM/labels/atl08.005/tanana'
main_dir_senegal = '/explore/nobackup/projects/ilab/projects/AIML_CHM/CNN_CHM/labels/atl08.005/senegal_20m'

In [ ]:
#!pip install segmentation-models

In [ ]:
def append_gdf(start, end, main_dir):
    atl08_gdf = []
    for year in range(start, end):
        atl08_gdf.append(atl08_io(main_dir, str(year), do_pickle=False))
    atl08_gdf = pd.concat(atl08_gdf)
    atl08_gdf.info()
    return atl08_gdf

In [ ]:
atl08_gdf_serc = append_gdf(2018, 2023, main_dir_serc)
atl08_gdf_howland = append_gdf(2018, 2023, main_dir_howland)
atl08_gdf_tanana = append_gdf(2018, 2023, main_dir_tanana)
atl08_gdf_senegal = append_gdf(2018, 2023, main_dir_senegal)

In [ ]:
atl08_gdf_serc =  atl08_gdf_serc.loc[atl08_gdf_serc["h_can_20m"] < 250]
atl08_gdf_howland = atl08_gdf_howland.loc[atl08_gdf_howland["h_can_20m"] < 250]
atl08_gdf_tanana = atl08_gdf_tanana.loc[atl08_gdf_tanana["h_can_20m"] < 250]
atl08_gdf_senegal = atl08_gdf_senegal.loc[atl08_gdf_senegal["h_can_20m"] < 250]

In [ ]:
# transpose and plot
ax = atl08_gdf_senegal["h_can_20m"].hist(bins=100, figsize=(7, 6))
ax.set_ylabel('Probability', fontsize=12)
ax.set_xlabel('Height (m)', fontsize=12)
ax.set_title('Histogram of ICESat-2 ATL08 Heights: Senegal')
ax.set_xlim((0, 60))
plt.show()
fig = ax.get_figure()
fig.savefig('icesat-height-senegal.png', bbox_inches='tight')

In [ ]:
# transpose and plot
ax = atl08_gdf_tanana["h_can_20m"].hist(bins=100, figsize=(7, 6))
ax.set_ylabel('Probability', fontsize=12)
ax.set_xlabel('Height (m)', fontsize=12)
ax.set_title('Histogram of ICESat-2 ATL08 Heights: Tanana, Alaska')
ax.set_xlim((0, 60))
plt.show()
fig = ax.get_figure()
fig.savefig('icesat-height-tanana.png', bbox_inches='tight')

In [ ]:
# transpose and plot
ax = atl08_gdf_howland["h_can_20m"].hist(bins=100, figsize=(7, 6))
ax.set_ylabel('Probability', fontsize=12)
ax.set_xlabel('Height (m)', fontsize=12)
ax.set_title('Histogram of ICESat-2 ATL08 Heights: Howland')
ax.set_xlim((0, 60))
plt.show()
fig = ax.get_figure()
fig.savefig('icesat-height-howland.png', bbox_inches='tight')

In [ ]:
# transpose and plot
ax = atl08_gdf_serc["h_can_20m"].hist(bins=100, figsize=(7, 6))
ax.set_ylabel('Probability', fontsize=12)
ax.set_xlabel('Height (m)', fontsize=12)
ax.set_title('Histogram of ICESat-2 ATL08 Heights: SERC')
ax.set_xlim((0, 60))
plt.show()
fig = ax.get_figure()
fig.savefig('icesat-height-serc.png', bbox_inches='tight')

In [ ]:
print("Serc: ", atl08_gdf_serc.total_bounds, atl08_gdf_serc.crs, atl08_gdf_serc.shape)
print("Howland: ", atl08_gdf_howland.total_bounds, atl08_gdf_howland.crs, atl08_gdf_howland.shape)
print("Tanana: ", atl08_gdf_tanana.total_bounds, atl08_gdf_tanana.crs, atl08_gdf_tanana.shape)
print("Senegal: ", atl08_gdf_senegal.total_bounds, atl08_gdf_senegal.crs, atl08_gdf_senegal.shape)

In [ ]:
def MAP_ATL08(
            ATL08_GDF_SUBSET,
            SUBSET_NAME,
            YEAR,
            DIR_MAP,
            TITLE="Canopy height from ICESat-2",
            MAP_COL='h_can',
            YEAR_COL='y',
            proj="+proj=laea +lat_0=60 +lon_0=-100 +x_0=90 +y_0=0 +ellps=GRS80"
        ):

    # Plot obs from night and day
    # My cmap
    forest_ht_cmap = LinearSegmentedColormap.from_list(
        'forest_ht',
        [
            '#636363', '#fc8d59', '#fee08b', '#ffffbf',
            '#d9ef8b', '#91cf60', '#1a9850', '#005a32'
        ],
        12
    )

    print(f"There are {ATL08_GDF_SUBSET.shape[0]} ATL08 observations.")
    print(len(ATL08_GDF_SUBSET.columns))
    xmin, ymin, xmax, ymax = ATL08_GDF_SUBSET.total_bounds

    # Define a projection for the maps and the geodataframe

    # Clip world to ATL08 gdf
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    world_atl08 = world.cx[xmin:xmax, ymin:ymax]

    atl08_gdf_chull = ATL08_GDF_SUBSET.unary_union.convex_hull

    world_atl08 = world[world.intersects(atl08_gdf_chull)]
    # NA = world[world['continent'] == 'North America'].to_crs(boreal_alb)

    atl08_gdf_proj = ATL08_GDF_SUBSET.to_crs(proj)
    world_atl08_proj = world_atl08.to_crs(proj)

    if 'all' in YEAR:
        YEAR = f'{ATL08_GDF_SUBSET[YEAR_COL].min()}' + \
            f'- {ATL08_GDF_SUBSET[YEAR_COL].max()}'
    ax_map_title = f'{TITLE}: {YEAR}'
    cbar_map_title = f'Canopy height [m] (ATL08: {MAP_COL})'

    d = datetime.date.today().strftime("%Y%b%d")
    # Set up correct number of subplots, space them out.
    fig, ax = plt.subplots(1, 1, figsize=(14, 10), sharex=True, sharey=True)

    bbox = atl08_gdf_proj.total_bounds

    world_atl08_proj.plot(
        ax=ax, facecolor='grey', edgecolor='black',  alpha=0.5)
    hb = ax.hexbin(
        atl08_gdf_proj.geometry.x, atl08_gdf_proj.geometry.y,
        C=atl08_gdf_proj[MAP_COL],
        reduce_C_function=np.median, gridsize=750,
        cmap=forest_ht_cmap, vmax=25, mincnt=1, alpha=0.7)
    world_atl08_proj.plot(
        ax=ax, facecolor='None', edgecolor='black',  alpha=0.9)

    cbar = plt.colorbar(
        hb, extend='max', spacing='proportional',
        orientation='vertical', shrink=0.7, format="%.0f")
    cbar.set_label(label=cbar_map_title, size=16)

    ax.set_xlim(bbox[[0, 2]])
    ax.set_ylim(bbox[[1, 3]])

    ax.set_title(ax_map_title, size=20, loc='left')
    ax.axis('off')
    ax.annotate(
        f'Filtered ATL08 obs: {len(ATL08_GDF_SUBSET)}',
        xy=(0.1, .08), xycoords='figure fraction',
        horizontalalignment='left',
        verticalalignment='top',
        fontsize=12,
        color='#555555'
    )
    fig_fn = os.path.join(
        DIR_MAP, 'atl08_alb_'+MAP_COL+'_'+SUBSET_NAME+'_'+YEAR+'_'+d+'.png')
    print(fig_fn)
    plt.savefig(fig_fn)


def MAP_ATL08_POLAR(atl08_gdf, YEAR=2021, MAP_COL='h_can'):

    cbar_map_title = f'Canopy height [m] (ATL08: {MAP_COL})'
    forest_ht_cmap = LinearSegmentedColormap.from_list(
        'forest_ht', [
            '#636363', '#fc8d59',
            '#fee08b', '#ffffbf',
            '#d9ef8b', '#91cf60',
            '#1a9850', '#005a32'
        ], 12
    )

    atl08_gdf = atl08_gdf.to_crs(ccrs.NorthPolarStereo())

    fig = plt.figure(figsize=[10, 5])
    ax1 = plt.subplot(1, 2, 1, projection=ccrs.NorthPolarStereo())
    ax2 = plt.subplot(1, 2, 2, projection=ccrs.NorthPolarStereo(),
                      sharex=ax1, sharey=ax1)
    fig.subplots_adjust(bottom=0.05, top=0.95,
                        left=0.04, right=0.95, wspace=0.02)

    # Limit the map to -60 degrees latitude and below.
    ax1.set_extent([-180, 180, 90, 45], ccrs.PlateCarree())

    ax1.add_feature(cartopy.feature.OCEAN)
    ax1.add_feature(cartopy.feature.LAND)

    ax2.add_feature(cartopy.feature.OCEAN, color='white')
    ax2.add_feature(cartopy.feature.LAND, color='grey')

    hb = ax2.hexbin(
        atl08_gdf.geometry.x,
        atl08_gdf.geometry.y,
        C=atl08_gdf[MAP_COL],
        reduce_C_function=np.median,
        gridsize=750,
        cmap=forest_ht_cmap,
        vmax=25, mincnt=1, alpha=0.7
    )

    ax1.gridlines(color='black')
    ax2.gridlines(color='black')

    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)

    ax2.set_boundary(circle, transform=ax2.transAxes)

    cbar = plt.colorbar(
        hb,
        extend='max',
        spacing='proportional',
        orientation='vertical',
        shrink=0.7,
        format="%.0f"
    )
    cbar.set_label(label=cbar_map_title, size=16)

    plt.show()


basemaps = {
    'Google Terrain': TileLayer(
        tiles='https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr='Google',
        name='Google Terrain',
        overlay=False,
        control=True
    ),
    'basemap_gray': TileLayer(
        tiles=(
            'http://services.arcgisonline.com/ArcGIS/rest/services/'
            'Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}'
        ),
        opacity=1,
        name="World gray basemap",
        attr="ESRI",
        overlay=False
    ),
    'Imagery': TileLayer(
        tiles=(
            'https://services.arcgisonline.com/arcgis/rest/services/'
            'World_Imagery/MapServer/tile/{z}/{y}/{x}'
        ),
        opacity=1,
        name="World Imagery",
        attr="ESRI",
        overlay=False
    ),
    'ESRINatGeo': TileLayer(
        tiles=(
            'https://server.arcgisonline.com/ArcGIS/rest/services/'
            'NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}'
        ),
        opacity=1,
        name='ESRI NatGeo',
        attr='ESRI',
        overlay=False
    )
}


def ADD_ATL08_OBS_TO_MAP(
            atl08_gdf,
            MAP_COL,
            DO_NIGHT,
            NIGHT_FLAG_NAME,
            foliumMap,
            RADIUS=10
        ):

    pal_height_cmap = cm.LinearColormap(
        colors=[
            'black',
            '#636363',
            '#fc8d59',
            '#fee08b',
            '#ffffbf',
            '#d9ef8b',
            '#91cf60',
            '#1a9850'
        ], vmin=0, vmax=25
    )
    pal_height_cmap.caption = f'Vegetation height from  ATL08 ({MAP_COL})'

    night_flg_label = 'day/night'
    if DO_NIGHT:
        atl08_gdf = atl08_gdf[
            atl08_gdf[NIGHT_FLAG_NAME] == 1]
        night_flg_label = 'night'
    print(f'Mapping {len(atl08_gdf)} {night_flg_label} ATL08 of {MAP_COL}')

    # https://stackoverflow.com/questions/61263787/folium-featuregroup-in-python
    # feature_group = folium.FeatureGroup('ATL08')

    atl08_cols_zip_list = [atl08_gdf.lat, atl08_gdf.lon, atl08_gdf[MAP_COL]]

    for lat, lon, ht in zip(*atl08_cols_zip_list):
        ATL08_obs_night = CircleMarker(
            location=[lat, lon],
            radius=RADIUS,
            weight=0.75,
            tooltip=str(round(ht, 2)) + " m",
            fill=True,
            # fill_color=getfill(h_can),
            color=pal_height_cmap(ht),
            opacity=1,
            name=f"ATL08 {night_flg_label} obs"
        )
        ATL08_obs_night.add_to(foliumMap)
    foliumMap.add_child(pal_height_cmap)
    # LayerControl().add_to(foliumMap)
    return foliumMap


def ADD_ATL03_OBS_TO_MAP(atl03_gdf, foliumMap):
    for lat, lon, phclassname, phcolor, elev in \
        zip(
            atl03_gdf.lat,
            atl03_gdf.lon,
            atl03_gdf['class_name'],
            atl03_gdf['color'],
            atl03_gdf['elev']
            ):
        ATL03_obs = CircleMarker(
            location=[lat, lon],
            radius=5,
            weight=1,
            tooltip=phclassname + ": "+str(round(elev, 2)) + " m",
            fill=True,
            # fill_color=getfill(h_can),
            color=phcolor,
            opacity=1,
            name="ATL03 classified photons"
        )
        ATL03_obs.add_to(foliumMap)

    return foliumMap


def MAP_LAYER_FOLIUM(
        LAYER=None,
        LAYER_COL_NAME=None,
        fig_w=1000,
        fig_h=400,
        lat_start=60,
        lon_start=-120,
        zoom_start=8):

    # Map the Layers
    Map_Figure = Figure(width=fig_w, height=fig_h)
    foliumMap = Map(
        tiles=None,
        location=(lat_start, lon_start),
        zoom_start=zoom_start,
        control_scale=True
    )
    Map_Figure.add_child(foliumMap)

    if LAYER is not None:
        GEOJSON_LAYER = GeoJson(
            LAYER,
            name='footprints',
            style_function=lambda x:
                {
                    'fillColor': 'gray',
                    'color': 'red',
                    'weight': 0.75,
                    'opacity': 1,
                    'fillOpacity': 0.5
                },
                tooltip=features.GeoJsonTooltip(
                    fields=[LAYER_COL_NAME],
                    aliases=[f'{LAYER_COL_NAME}:'],
                )
        )
        GEOJSON_LAYER.add_to(foliumMap)

    basemaps['Imagery'].add_to(foliumMap)
    basemaps['basemap_gray'].add_to(foliumMap)
    basemaps['ESRINatGeo'].add_to(foliumMap)

    LayerControl().add_to(foliumMap)
    plugins.Geocoder().add_to(foliumMap)
    plugins.MousePosition().add_to(foliumMap)
    minimap = plugins.MiniMap()
    plugins.Fullscreen().add_to(foliumMap)
    foliumMap.add_child(minimap)

    return foliumMap


def MAP_FOLIUM(
            ADD_LAYER=False,
            LAYER_FN=None,
            basemaps=basemaps,
            fig_w=1000,
            fig_h=400,
            lat_start=5,
            lon_start=-17,
            zoom_start=8,
            LAYER_NAME="HRSI CHM footprints"
        ):
    Map_Figure = Figure(width=fig_w, height=fig_h)
    foliumMap = Map(
        location=(lat_start, lon_start),
        zoom_start=zoom_start, control_scale=True, tiles=None
    )
    Map_Figure.add_child(foliumMap)

    basemaps['Imagery'].add_to(foliumMap)
    basemaps['basemap_gray'].add_to(foliumMap)
    basemaps['ESRINatGeo'].add_to(foliumMap)

    if ADD_LAYER:
        lyr = gpd.read_file(LAYER_FN)
        lyrs = gpd.GeoDataFrame(
            pd.concat([lyr], ignore_index=True))
        lyr_style = {
            'fillColor': 'gray',
            'color': 'red',
            'weight': 0.75,
            'opacity': 1,
            'fillOpacity': 0.5}
        GeoJson(
            lyrs, name=LAYER_NAME,
            style_function=lambda x: lyr_style).add_to(foliumMap)

    LayerControl().add_to(foliumMap)
    plugins.Geocoder().add_to(foliumMap)
    plugins.MousePosition().add_to(foliumMap)
    minimap = plugins.MiniMap()
    plugins.Fullscreen().add_to(foliumMap)
    foliumMap.add_child(minimap)

    return foliumMap


def MAP_ATL08_FOLIUM(
            atl08_gdf,
            MAP_COL='h_can',
            DO_NIGHT=True,
            NIGHT_FLAG_NAME='night_flg',
            ADD_LAYER=True,
            LAYER_FN=None,
            basemaps=basemaps,
            fig_w=1000,
            fig_h=400,
            RADIUS=10
        ):

    if LAYER_FN is None:
        ADD_LAYER = False

    # Map the Layers
    Map_Figure = Figure(width=fig_w, height=fig_h)

    # ------------------
    foliumMap = Map(
        location=(atl08_gdf.lat.mean(), atl08_gdf.lon.mean()),
        zoom_start=8, control_scale=True, tiles=None
    )
    Map_Figure.add_child(foliumMap)

    basemaps['Imagery'].add_to(foliumMap)
    basemaps['basemap_gray'].add_to(foliumMap)
    basemaps['ESRINatGeo'].add_to(foliumMap)

    if ADD_LAYER:
        lyr = gpd.read_file(LAYER_FN)
        lyrs = gpd.GeoDataFrame(
            pd.concat([lyr], ignore_index=True))
        lyr_style = {
            'fillColor': 'gray',
            'color': 'gray',
            'weight': 0.75,
            'opacity': 1,
            'fillOpacity': 0.5
        }
        GeoJson(
            lyrs,
            name="HRSI CHM footprints",
            style_function=lambda x: lyr_style
        ).add_to(foliumMap)

    foliumMap = ADD_ATL08_OBS_TO_MAP(
        atl08_gdf, MAP_COL=MAP_COL, DO_NIGHT=DO_NIGHT,
        NIGHT_FLAG_NAME=NIGHT_FLAG_NAME, foliumMap=foliumMap, RADIUS=RADIUS)
    # foliumMap.add_child(LayerControl()) #LayerControl().add_to(foliumMap)

    LayerControl().add_to(foliumMap)
    # Add fullscreen button
    plugins.Fullscreen().add_to(foliumMap)
    # plugins.Geocoder().add_to(foliumMap)
    plugins.MousePosition().add_to(foliumMap)
    minimap = plugins.MiniMap()
    foliumMap.add_child(minimap)

    return foliumMap


def MAP_ATL03_FOLIUM(
            atl03_gdf,
            basemaps=basemaps,
            fig_w=1000,
            fig_h=400,
            zoom_start=10
        ):

    # Map the Layers
    Map_Figure = Figure(
        width=fig_w, height=fig_h)

    foliumMap = Map(
        # tiles="Stamen Terrain",
        location=(atl03_gdf.lat.mean(), atl03_gdf.lon.mean()),
        zoom_start=zoom_start, control_scale=True, tiles=None
    )
    Map_Figure.add_child(foliumMap)

    foliumMap = ADD_ATL03_OBS_TO_MAP(atl03_gdf, foliumMap)

    basemaps['Imagery'].add_to(foliumMap)
    basemaps['basemap_gray'].add_to(foliumMap)
    LayerControl().add_to(foliumMap)
    plugins.Geocoder().add_to(foliumMap)
    plugins.MousePosition().add_to(foliumMap)
    minimap = plugins.MiniMap()
    foliumMap.add_child(minimap)

    return foliumMap


def GET_FOOTPRINT(r_fn, out_gpkg_fn):
    # TODO get this from raster
    dx = 1
    XY = [504675.55, 7695881.9]
    nx = 1000
    west = XY[0] - (nx * dx)/2
    north = XY[1] + (nx * dx)/2
    Transform = from_origin(west, north, dx, dx)

    # TODO read r_fn with rasterio
    Array = np.zeros(shape=(10, 10))
    Array[2:4, 2:4] = 1
    Array[6:9, 6:7] = 2

    d = {}
    d['val'] = list()
    geometry = list()

    for shp, val in features.shapes(
            Array.astype('int16'), transform=Transform):
        d['val'].append(val)
        geometry.append(shape(shp))
        print('%s: %s' % (val, shape(shp)))
    df = pd.DataFrame(data=d)

    # TODO get crs from raster
    geo_df = GeoDataFrame(
        df, crs={'init': 'EPSG:32608'}, geometry=geometry)
    geo_df['area'] = geo_df.area
    geo_df.to_file(
        'JustSomeRectanglesInTheNWT.shp', driver='GPKG')  # 'ESRI Shapefile'

# TODO: add histograms here
# TODO: add dashboards here


In [ ]:
%%time
SAMP_FRAC=0.25
MAP_ATL08_FOLIUM(
    atl08_gdf_serc.sample(frac=SAMP_FRAC), MAP_COL='h_can_20m',
    DO_NIGHT=False, LAYER_FN=None, RADIUS=3
)

In [ ]:
%%time
SAMP_FRAC=0.25
MAP_ATL08_FOLIUM(
    atl08_gdf_howland.sample(frac=SAMP_FRAC), MAP_COL='h_can_20m',
    DO_NIGHT=False, LAYER_FN=None, RADIUS=3
)

In [ ]:
#%%time
#SAMP_FRAC=0.25
#MAP_ATL08_FOLIUM(
#    atl08_gdf_tanana.sample(frac=SAMP_FRAC), MAP_COL='h_can_20m',
#    DO_NIGHT=False, LAYER_FN=None, RADIUS=3
#)

In [ ]:
%%time
SAMP_FRAC=0.25
MAP_ATL08_FOLIUM(
    atl08_gdf_senegal.sample(frac=SAMP_FRAC), MAP_COL='h_can_20m',
    DO_NIGHT=False, LAYER_FN=None, RADIUS=3
)

In [ ]:
from vhr_cnn_chm.config import CHMConfig as Config
from vhr_cnn_chm.utils import get_atl08_gdf, get_wv_evhr_gdf, \
    filter_gdf_by_list, filter_polygon_in_raster_parallel, \
    extract_tiles_parallel
from tensorflow_caney.utils.system import seed_everything

In [ ]:
wv_data_regex = '/explore/nobackup/projects/ilab/projects/AIML_CHM/CNN_CHM/data/WorldView/SRLite/Alaska/*M1BS*.tif'
general_crs = "EPSG:4326"
atl08_start_year = 2018
atl08_end_year = 2022

In [ ]:
# Read WorldView and get boundaries from file
wv_evhr_gdf = get_wv_evhr_gdf(
    wv_data_regex, crs=None)

print(f'Load WorldView GDF, {wv_evhr_gdf.shape[0]} rows.')
print(f'WorldView Y:, {wv_evhr_gdf["wv_year"].unique()} rows.')
print(wv_evhr_gdf.crs, atl08_gdf_tanana.crs)

# Filter GDF by year range
#wv_evhr_gdf = filter_gdf_by_list(
#    wv_evhr_gdf, 'wv_year', list(
#        range(atl08_start_year, atl08_end_year))
#)
print(f'Filtered WorldView GDF, {wv_evhr_gdf.shape[0]} rows.')
print(f'WorldView Y:, {wv_evhr_gdf["wv_year"].unique()} rows.')

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

ax.set_aspect('equal')

#wv_evhr_gdf.plot(ax=ax, color='white', edgecolor='black', figsize=(10, 20))
#atl08_gdf_tanana.plot(ax=ax, marker='o', color='red', markersize=5, figsize=(10, 20))
#plt.show();

# print(atl08_gdf_tanana.total_bounds)

atl08_gdf_tanana = atl08_gdf_tanana.to_crs(32605)
print(wv_evhr_gdf.crs, atl08_gdf_tanana.crs)


wv_evhr_gdf.to_file("wv_tanana_long_v3.gpkg", layer='layer', driver="GPKG")
atl08_gdf_tanana.to_file("icesat_tanana_v3.gpkg", layer='layer', driver="GPKG")



#base = wv_evhr_gdf.plot(color='white', edgecolor='black', figsize=(10, 20))
#atl08_gdf_tanana.plot(ax=base, marker='o', color='red', markersize=5);

Misc info:
https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb

In [ ]:
# glight_intersect = '/adapt/nobackup/people/zwwillia/dataCollections/GLiHT/SR_lite_intersect.gpkg'
# glight_gdf = gpd.read_file(glight_intersect)

In [ ]:
# glight_gdf['location'][0]

In [ ]:
#%%time
#SAMP_FRAC=0.25
#maplib.MAP_ATL08_FOLIUM(
#    glight_gdf.sample(frac=SAMP_FRAC), MAP_COL='area_ha',
#    DO_NIGHT=False, LAYER_FN=None, RADIUS=3
#)

In [ ]:
regex = '/explore/nobackup/projects/ilab/projects/AIML_CHM/CNN_CHM/development/senegal/metadata/intersection/*/*.gpkg'
filenames = glob(regex)
len(filenames)

In [ ]:
filenames[0]

In [ ]:
gdf = gpd.read_file(filenames[0])
gdf['scene_id'].iloc[0]

#/explore/nobackup/projects/ilab/projects/AIML_CHM/CNN_CHM/development/senegal/metadata/intersection/CAS/WV02_20180111_M1BS_1030010076791B00-toa.gpkg

In [ ]:
raster_data = rasterio.open(gdf['scene_id'].iloc[0])
#raster_data = rasterio.open('/explore/nobackup/projects/3sl/products/cloudmask/v1/CAS/WV02_20200615_M1BS_10300100A8B6BF00-toa.cloudmask.tif')

#arr = raster_data.read(1)
#print(np.count_nonzero(arr == 0))
#print(np.count_nonzero(arr == 1))

# atl08_data = gpd.read_file(atl08_filename, layer='ATL08_WorldView')
# print(wv_filename)

fig, ax = plt.subplots(figsize=(20,15))

# transform rasterio plot to real world coords
extent=[raster_data.bounds[0], raster_data.bounds[2], raster_data.bounds[1], raster_data.bounds[3]]
ax = rasterio.plot.show(raster_data, extent=extent, ax=ax, cmap='pink')
gdf.plot(ax=ax)

plt.title(f'{gdf["scene_id"].iloc[0]}, {gdf.shape[0]}')
plt.show()

In [ ]:
images = sorted(glob('/explore/nobackup/projects/ilab/projects/AIML_CHM/CNN_CHM/development/senegal/images/*.npy'))
labels = sorted(glob('/explore/nobackup/projects/ilab/projects/AIML_CHM/CNN_CHM/development/senegal/labels/*.npy'))

In [ ]:
plt.imshow(np.load(images[0])[:, :, 6]/10000)
plt.show()
plt.imshow(np.squeeze(np.load(labels[0])))
plt.show()

In [ ]:
import xarray as xr
import h5py

In [ ]:
gedi = '/explore/nobackup/projects/ilab/projects/AIML_CHM/CNN_CHM/data/GEDI/CAS/WV02_20220106_M1BS_10300100CB9E5100-toa/GEDI02_A_2021323165429_O16638_02_T09575_02_003_02_V002.h5'
wv = '/explore/nobackup/projects/3sl/data/VHR/CAS/M1BS/WV02_20220106_M1BS_10300100CB9E5100-toa.tif'

In [ ]:
#gedi_arr = xr.open_dataset(gedi)
gediL2A = h5py.File(gedi, 'r')
list(gediL2A.keys())

In [ ]:
#list(gedi_arr['BEAM0000'].keys())
beamNames = [g for g in gediL2A.keys() if g.startswith('BEAM')]
for b in beamNames: 
    print(f"{b} is a {gediL2A[b].attrs['description']}")

In [ ]:
gediL2A['BEAM0101']['rh']

In [ ]:
list(gediL2A['BEAM0101'].keys())

In [ ]:
gediL2A['BEAM0101']['shot_number']

In [ ]:
gediL2A['BEAM0101']['rh'][0]

In [ ]:
lonSample, latSample, shotSample, qualitySample, beamSample, rh90 = [], [], [], [], [], []  # Set up lists to store data
beamName = 'BEAM0101'

# Open the SDS
lats = gediL2A[f'{beamName}/lat_lowestmode'][()]
lons = gediL2A[f'{beamName}/lon_lowestmode'][()]
shots = gediL2A[f'{beamName}/shot_number'][()]
quality = gediL2A[f'{beamName}/quality_flag'][()]

# Take every 100th shot and append to list
for i in range(len(shots)):
    #if i % 100 == 0:
    shotSample.append(str(shots[i]))
    lonSample.append(lons[i])
    latSample.append(lats[i])
    qualitySample.append(quality[i])
    beamSample.append(beamName)
    rh90.append(gediL2A['BEAM0101']['rh'][i][89])

# Write all of the sample shots to a dataframe
latslons = pd.DataFrame({'Beam': beamSample, 'Shot Number': shotSample, 'Longitude': lonSample, 'Latitude': latSample,
                         'Quality Flag': qualitySample, 'rh90': rh90})
latslons


del beamSample, quality, qualitySample, gediL2A_objs, latSample, lats, lonSample, lons, shotSample, shots 
latslons['geometry'] = latslons.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)
# Convert to a Geodataframe
latslons = gp.GeoDataFrame(latslons)
latslons = latslons.drop(columns=['Latitude','Longitude'])
latslons['geometry']

In [ ]:
!/explore/nobackup/projects/ilab/projects/AIML_CHM/CNN_CHM/data/WorldView/